# DBSCAN Paralelo para deteccón de ruido

Mateo De La Roche - 190748

Bruno Molina Zacatenco - 187228

El algoritmo dbscan permite encontrar clusters o cúmulos y detectar ruido en un conjunto de puntos. Los únicos parámetros que necesita son epsilon y minPts. El primero es la distancia máxima entre dos puntos para que sean considerados vecinos y el segundo es el número mínimo de puntos que debe tener un vecindario para ser considerado un cluster.
Para este proyecto se implemento un "naive" dbscan ya que solo se encarga de encontrar el ruido y no de distinguir entre clusters. 

El objetivo del proyecto es comparar como mejora la eficiencia al paralelizar el algoritmo y ver como cambia según el número de hilos y el tamaño del dataset. Para hacer más facíl la generación se nos proporcióno el siguiento código:


In [2]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np

def points_generator(n_points):
    # randomly generating data points and noise 
    points, y_true = make_blobs(n_samples=n_points, 
                                centers=4, 
                                cluster_std=0.06, 
                                random_state=11, 
                                center_box=(0, 1.0))

    # only positive points and with three decimals
    points = np.round(np.abs(points[:, ::-1]), 3)

    # storing points into a csv file
    np.savetxt("data/"+str(n_points)+"_data.csv", points, delimiter=",",  fmt="%.3f")


Al tener los puntos generados, se implementaron tres versiones del algoritmo, una serial, una paralelizando el for que recorre los puntos y una paralelizando el for que recorre los vecinos.

Antes de pasar a las implementaciónes, para la distancia de los vecinos se utilizo la distancia euclidiana, calculada con la siguiente función:
```c++
float distance(float x1, float y1, float x2, float y2)
{   
    return sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2));
}
```

### Version Serial
 ```c++
void noise_detection(float** points, float epsilon, int min_samples, long long int size) {
    
    //recorremos todos los puntos
    for (long long int i=0; i < size; i++) {
        int vecinos = 0;
        //medimos la distancia de cada punto con todos los demas
        for (long long int j=0; j < size; j++) {
            if(j==i) continue;
            if (distance(points[i][0], points[i][1], points[j][0], points[j][1]) < epsilon) {
                vecinos++;
            }
            //si ya encontramos el minimo de vecinos, paramos    
            if (vecinos >= min_samples) {
                points[i][2] = 1;
                break;
            }
        
        }
    }      
}
```

### Version Paralelizada en el for de los puntos
```c++
void noise_detection(float** points, float epsilon, int min_samples, long long int size, int num_threads) {
    
    long long int i;
    int vecinos = 0;

    omp_set_num_threads(num_threads);

    #pragma omp parallel shared(points, epsilon, min_samples, size) private(i)
    {
        #pragma omp for reduction(+:vecinos)
        for (i=0; i < size; i++) 
        { 
            //medimos la distancia de cada punto con todos los demas
            for (long long int j=0; j < size; j++) {
                if(j==i) continue;
                if (distance(points[i][0], points[i][1], points[j][0], points[j][1]) < epsilon) {
                    vecinos++;
                }
                //si ya encontramos el minimo de vecinos, paramos    
                if (vecinos >= min_samples) {
                    points[i][2] = 1;
                    break;
                }
            
            }
        }   
    }   
}
```


### Version Paralelizada en el for de los vecinos
```c++
void noise_detection(float** points, float epsilon, int min_samples, long long int size, int num_threads) {
    
    long long int j;
    int vecinos = 0;
    volatile bool flag = false;

    omp_set_num_threads(num_threads);

    #pragma omp parallel shared(points, epsilon, min_samples, size, flag) private(j)
    {
        for (long long int i=0; i < size; i++) {
            //medimos la distancia de cada punto con todos los demas
            #pragma omp for reduction(+:vecinos)
            for (j=0; j < size; j++) 
            {
                if(flag) continue;
                if(j==i) continue;
                if (distance(points[i][0], points[i][1], points[j][0], points[j][1]) < epsilon) {
                    vecinos++;
                }
                //si ya encontramos el minimo de vecinos, paramos    
                if (vecinos >= min_samples) {
                    points[i][2] = 1;
                    flag = true;
                }
            
            }
        }   
    }   
}

Cabe mencionar que se nos proporcionaron funciónes para cargar el csv y para guardar el resultado. Para tener un mejor orden en los archivos, se les hicieron unas modificaciones para separar los datasets y los resultados en diferentes carpetas. 
```c++	
void load_CSV(string file_name, float** points, long long int size) {
    ifstream in("data/"+file_name);
    if (!in) {
        cerr << "Couldn't read file: " << file_name << "\n";
    }
    long long int point_number = 0; 
    while (!in.eof() && (point_number < size)) {
        char* line = new char[12];
        streamsize row_size = 12;
        in.read(line, row_size);
        string row = line;
        //cout << stof(row.substr(0, 5)) << " - " << stof(row.substr(6, 5)) << "\n";
        points[point_number][0] = stof(row.substr(0, 5));
        points[point_number][1] = stof(row.substr(6, 5));
        point_number++;
    }
}

//convierte la matriz de points en un archivo csv
void save_to_CSV(string file_name, float** points, long long int size) {
    fstream fout;
    fout.open("dump/"+file_name, ios::out);
    for (long long int i = 0; i < size; i++) {
        fout << points[i][0] << ","
             << points[i][1] << ","
             << points[i][2] << "\n";
    }
}
```

El main de las tres implementaciónes es prácticamente el mismo, solo cambia que para las versiones paralelizadas se le pasa el numero de hilos o threads como argumento y la tercer columna del csv la cual indica la implementación de esa ejecución. 
```c++
int main(int argc, char** argv) {

    const float epsilon = 0.03;
    const int min_samples = 10;

    const long long int size = atoi(argv[1]);

    const string input_file_name = to_string(size)+"_data.csv";
    const string output_file_name = to_string(size)+"serial_results.csv";    
    float** points = new float*[size];
    double start = 0;
    double end = 0;


    for(long long int i = 0; i < size; i++) {
        points[i] = new float[3]{0.0, 0.0, 0.0}; 
        // index 0: position x
        // index 1: position y 
        // index 2: 0 for noise point, 1 for core point
    }

    load_CSV(input_file_name, points, size);
        
    start = omp_get_wtime();
    noise_detection(points, epsilon, min_samples, size); 
    end = omp_get_wtime();

    cout << size << ",serial,1," << end - start << "\n";
                    //p_vecinos
                    //p_puntos

    save_to_CSV(output_file_name, points, size);


    for(long long int i = 0; i < size; i++) {
        delete[] points[i];
    }
    delete[] points;
    return 0;
}
```